<a href="https://colab.research.google.com/github/takedatmh/toyama/blob/main/QLoRA_LLM_FineTuning_for_Japanese_AI_Beginners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ▶︎ 【QLoRA】ローカル環境で使うための日本語LLMファインチューニング
---

# 【unsloth編：LLMのファインチューニング入門】<br>
このノートブックは、以下の<br><br>
・[unslothai/unsloth（Apache-2.0 license）| GitHub ](https://github.com/unslothai/unsloth?tab=readme-ov-file)<br><br>
という、大規模言語モデル（LLM：Large Language Model）を<br><br>

・「**QLoRA**」(Quantized Low-Rank Adaptation）<br>
＊メモリ効率化手法のLoRAに4ビット（NormalFloat - NF4など）以下の量子化を組み合わせた場合には「QLoRA」と呼ぶ<br><br>
でファインチューニング（微調整）するプログラムを活用して、ローカル環境で使える対話型生成AIアプリ（ソフトウェア）用に日本語LLMをファインチューニングした「**GGUF形式**」のLLMを保存するためのチュートリアルコードです。<br>
一連の情報が、LLMを自分好みにカスタマイズして活用したいと考えている方が、既存のLLMのカスタマイズし、今後の実践利用に向けて検証するために試行錯誤を始めるきっかけになることがありましたら幸いです。
<br><br>
<br><br>
**【Google Colaboratory上のコードの動かし方】**<br><br>
まず始めに、Google Colaboratory上のプログラムを実行するには、<br><br>
**①Googleアカウントでログイン**<br>
**②ドライブにコピーを保存**（QLoRA-LLM-FineTuning-for-Japanese-AI-Beginners.ipynbのノートブックを保存）<br>
＊Google Colaboratoryのメニューから「ファイル - ドライブにコピーを保存」
<br><br>
を実行し、お好きな名前に変更後に以下のコードを実行していきます。<br><br>
尚、このノートブックの初期設定は<br><br>
**・T4 GPU**<br>
＊Google Colaboratory無料枠で使えるGPU<br>
＊「**ランタイム - ランタイムのタイプを変更 - ハードウェア アクセラレータ**」で変更できます<br><br>
に設定してあります。<br>
<br><br>
**【チュートリアル動画】**<br><br>
2024年3月25日（月）にチュートリアル動画を公開しました。<br><br>
[【現代の魔法】QLoRA編：日本語LLMのファインチューニング & ローカル環境アプリで動かす方法  - Fine Tuning LLM & How to Use in Local App by RehabC - デジタルで、遊ぶ。（YouTube）](https://youtu.be/cBY36C1CQeU)<br>
視聴時間：53分16秒
<br><br>
文字情報だけでは、わかりにくい場合などにご活用いただけますと幸いです。
<br><br>

**【プログラムのライセンス】**
<br>**Apache-2.0 license**

©︎ 2024 child programmer<br><br>
unsloth is licensed under the Apache-2.0 license, Copyright © unsloth. All Rights Reserved.
<br><br>
<br><br>

#【ステップ１】ファインチューニング前のLLMで推論 編
---
後で回答を変化を確認しやすいように、ファインチューニング前のLLMで推論してみましょう。<br>
チュートリアルでは<br><br>
・「**指示チューニング**」<br>
（LLMの基盤モデルに質問と回答を学習させた事前学習済みモデル）<br><br>
・「**人間のフィードバックからの強化学習**」<br>
（RLHF：Reinforcement Learning from Human Feedback）
<br><br>
されている<br><br>
・[tokyotech-llm/Swallow-7b-instruct-hf | Hugging Face](https://huggingface.co/tokyotech-llm/Swallow-7b-instruct-hf)<br>
＊Llama2の日本語能力強化版のSwallowの7B（70億パラメータ）版の指示チューニングLLM<br>
＊「東京工業大学情報理工学院の岡崎研究室/横田研究室」「国立研究開発法人産業技術総合研究所」の研究チームにより開発されたLLM
<br>
注：使用するLLMによって、実行コードを微調整する必要があります。
<br><br>
を活用して、ファインチューニング前後の変化を比較していきます。
<br><br>



#【事前準備①】unslothをクローン&依存関係のインストール

In [ ]:
# @title 実行コード

# 2024年3月16日以降のコード（Google Colaboratoryのアップデートのため）
# %%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Ampere・Hopperなどの新型GPU (RTX 30xx, RTX 40xx, A100, H100, L40)の場合に以下を実行
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 旧型のGPU (V100, Tesla T4, RTX 20xx)の場合に以下を実行
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass


# 2024年3月16日時点の依存関係のバージョン
# datasets-2.18.0 dill-0.3.8 docstring-parser-0.16 multiprocess-0.70.16 shtab-1.7.1 tyro-0.7.3 unsloth-2024.3 xxhash-3.4.1
# accelerate-0.28.0 bitsandbytes-0.43.0 peft-0.9.0 trl-0.7.11 xformers-0.0.25


# -2024年3月15日までのコード
# # 出力結果を非表示化
# %%capture
# import torch
# major_version, minor_version = torch.cuda.get_device_capability()
# if major_version >= 8:
#     # Ampere・Hopperなどの新型GPU (RTX 30xx, RTX 40xx, A100, H100, L40)の場合に以下を実行
#     !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"
# else:
#     # 旧型のGPU (V100, Tesla T4, RTX 20xx)の場合に以下を実行
#     !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
# pass


<br><br>

#【事前準備②】LLMをダウンロード
LLMの一例
    # 日本語継続事前学習済みモデル - 基盤モデル（2023年12月公開）
    tokyotech-llm/Swallow-7b-hf

    #「Swallow-7b-hf」に比べて、より多くの日本語トークンで学習された
    # 日本語継続事前学習済みモデル - 基盤モデル（2024年3月公開）
    tokyotech-llm/Swallow-7b-plus-hf
    
    #「Mistral-7B-v0.1」で学習された
    # 日本語継続事前学習済みモデル – 基盤モデル（2024年3月公開）
    # 7Bモデルだが、Swallow 13Bに迫る日本語性能とのこと
    tokyotech-llm/Swallow-MS-7b-v0.1

    #「Swallow-7b-hf」の指示チューニングモデル（2023年12月公開）
    tokyotech-llm/Swallow-7b-instruct-hf
    
今回は、ローカル環境でファインチューニング後のLLMを使うために「**instruction**」「**input**」「**output**」からなるAlpaca形式で学習された指示チューニングされた「**Swallow-7b-instruct-hf**」のLLMを利用します。<br><br><br>
**【Hugging Faceで現在公開されている日本語LLMをチェック】**<br>
[キーワード「japanese」（トレンド順）- Models | Hugging Face](https://huggingface.co/models?sort=trending&search=japanese)<br>
**補足説明：**
モデル名で散見する「**b**」というのはモデルのパラメータ（重み）が「**billion：10億**」という意味です。<br>
＊1.4B：14億パラメータ<br>
＊1.7B：17億パラメータ<br>
＊3.6B：36億パラメータ<br>
＊7B：70億パラメータ<br>
＊13B：130億パラメータ<br>
＊70B：700億パラメータ<br>
<br><br>
試した範囲では、東京大学・松尾研究室発のAIカンパニーからリリースされていることで有名なLlama形式で学習されているELYZA（イライザ）の<br><br>
[elyza/ELYZA-japanese-Llama-2-7b-fast-instruct | Hugging Face](https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast)<br><br>
のLLMの場合には、このノートブックでもファインチューニングして推論までできましたが、ローカル環境のアプリで動かそうとするとエラー（アプリがクラッシュ）となるようでした。<br><br>
（LLMのダウンロードに3〜16分ほどかかります）

In [ ]:
# @title 実行コード
# LLMダウンロード
from unsloth import FastLanguageModel
import torch

# 最大シーケンス長の指定
# 内部でRoPEスケーリングを自動サポート
max_seq_length = 2048
# データ型dtypeの指定。「None」で自動検出
dtype = None
# 4-bit量子化。メモリ使用量を減らすために4bit量子化を使用する場合には「True」。「False」でもよい。
load_in_4bit = True

# 英語版ですが以下の4-bit量子化モデルも指定できるようです
# その他の4-bit量子化のLLMは https://huggingface.co/unsloth へ
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
]

# LLMの指定など
model, tokenizer = FastLanguageModel.from_pretrained(
    # @markdown  ファインチューニングを実行するLLMを指定します。
    model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param {type:"string"}
    ,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #「hf」のモデルで利用。その他の場合にはコメントアウト「#」などで無効化
    #「hf：RLHF - Reinforcement Learning from Human Feedback
    # 人間のフィードバックからの強化学習
    token = "hf_...",
)


# 推論①：指示形式

In [ ]:
# @title 実行コード

alpaca_prompt = """以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。\n\n

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)
# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt = "こんにちは。" # @param {type:"string"}
inputs = tokenizer(
[
    alpaca_prompt.format(
        # instruction：プロンプトとして活用しています。
         input_prompt,
        # input：必要に応じて入力します。
        "",
        # output：文章を生成するにはここを空欄にします。
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64) #「max_new_tokens = 」の値で出力のトークン数を指定できます

In [ ]:
# @title 実行コード

alpaca_prompt = """以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。\n\n

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)
# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt = "YouTubeのRehabC – デジタルで、遊ぶ。チャンネルについて教えてください。" # @param {type:"string"}
inputs = tokenizer(
[
    alpaca_prompt.format(
        # instruction：プロンプトとして活用しています。
         input_prompt,
        # input：必要に応じて入力します。
        "",
        # output：文章を生成するにはここを空欄にします。
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64) #「max_new_tokens = 」の値で出力のトークン数を指定できます

# 推論②：対話型・チャット形式テンプレート

In [ ]:
# @title 実行コード
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    # サポートしているテンプレート「zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth」
    chat_template = "alpaca",
    # ShareGPTスタイル
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    # <|im_end|>を</s>に変換しマッピング
    map_eos_token = True,
)

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)

# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt_chat = "こんにちは。" # @param {type:"string"}
messages = [
    {"from": "human", "value": input_prompt_chat},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True) #「max_new_tokens = 」の値で出力のトークン数を指定できます
tokenizer.batch_decode(outputs)

In [ ]:
# @title 実行コード
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    # サポートしているテンプレート「zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth」
    chat_template = "alpaca",
    # ShareGPTスタイル
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    # <|im_end|>を</s>に変換しマッピング
    map_eos_token = True,
)

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)

# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt_chat = "YouTubeのRehabC – デジタルで、遊ぶ。チャンネルについて教えてください。" # @param {type:"string"}
messages = [
    {"from": "human", "value": input_prompt_chat},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

<br>
<br>
<br>

# 【ステップ２】LLMのQLoRAファインチューニング 編
---
それでは、ファインチューニングを始めてみましょう。<br>
チュートリアルでは、LLMに特定の知識（今回は、特定のYouTubeチャンネル）<br><br>
・[RehabC - デジタルで、遊ぶ。（YouTube）](https://www.youtube.com/channel/UCR04rlF8TZ-xCH2wJvp9WOw)
<br><br>
を教えてあげます。<br><br>


#【事前準備①：データセットの準備】
（自作カスタムデータセット・Huging Face公開データセット対応）
<br>
<br>
自分で用意したオリジナルデータセットや、Hugging Faceに公開されているデータセットで学習してみましょう。<br>
自作データセットを使う場合には、CSV形式ファイルをGoogle Colaboratoryにアップロード後に、以下の「**実行コード - A**」を実行します。<br>
また、Hugging Faceの公開データセットを使う場合には、データセット名を指定後に、以下の「**実行コード - B**」を実行します。<br>
チュートリアルでは<br><br>
**・同じ回答に対する様々なバリエーションの質問など**<br>
（学習用19データ）<br><br>
を並べた自作データセットを使って、新たな特定の知識などを学習させてみます。
<br><br><br>
**【データセットの文章例】**<br>
**instruction：**<br>
YouTubeのRehabC – デジタルで、遊ぶ。チャンネルについて教えてください。<br><br>
**input：**<br>
（空欄）<br><br>
**output：**<br>
RehabCチャンネルは、2014年に開設されたデジタルテクノロジー教育系チャンネルです。
<br><br>
＊　試した範囲の情報では、今回の7BのLLMの場合には、1つの知識を教えるのに、様々なバリエーションに富んだ質問方式にして、同じ回答のデータセットを作成した方が良さそうでした。また、間違える傾向にある質問に関しては、他の質問よりも学習させる数（行）を増やすと正答率が上がる印象を受けています。
<br><br><br>

**【データセットのテンプレート】**<br>
：[ダウンロード - 【QLoRA編】LLMファインチューニング用データセットテンプレート by 子供プログラマー](https://child-programmer.com/download/llm-ft-qlora-tutorial/)
<br>
＊データセットのテンプレートをエクセル（Windowsの方）やNumbers（Macの方）で編集後に「**dataset**」（dataset.csv）という名前のCSV形式ファイルで書き出してください
<br><br>
**【チュートリアル用サンプルデータセット】**<br>
：[ダウンロード - サンプル： 【QLoRA編】日本語LLMファインチューニング用データセット by 子供プログラマー](https://child-programmer.com/download/llm-ft-qlora-tutorial-sample/)<br>
＊ダウンロード時点のファイル名は「**sample_dataset**」（sample_dataset.csv）ですが、利用する際には「**dataset**」（dataset.csv）という名前に変更してご利用ください
<br><br>

**【参考情報】**<br>
推奨されるデータセットの量：1000～50000<br>
[Documentation - FAQs：How much data is generally required to fine-tune a model? | H2O.AI](https://h2oai.github.io/h2o-llmstudio/faqs#how-much-data-is-generally-required-to-fine-tune-a-model)<br>
（大規模言語モデルをファインチューニングするには一般的にどのくらいのデータ量が必要ですか?）<br><br>

In [ ]:
# @title 実行コード - A（自作のカスタムデータセット版）
alpaca_prompt = """以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。\n\n

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        #「EOS_TOKEN」を追加
        # 生成が永遠に続いてしまうため
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# データセットを処理するために「load_dataset」をインポート
from datasets import load_dataset

# データセットの設定（自作データセット）の設定 - はじめ
# @markdown  データセットのファイルのパス（CSV形式のデータセット）を指定します。
dataset_files = "/content/dataset.csv" # @param {type:"string"}
dataset = load_dataset("csv", data_files = dataset_files, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
# データセットの設定（自作データセット）の設定 - おわり



<br><br>

In [ ]:
# @title 実行コード - B（Huging Face公開データセット版）
alpaca_prompt = """以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。\n\n

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        #「EOS_TOKEN」を追加
        # 生成が永遠に続いてしまうため
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# データセットを処理するために「load_dataset」をインポート
from datasets import load_dataset


#↓Hugging Faceの既存のデータセットを使いたい場合には以下のコードを参考に
# コードをカスタマイズしてみてください

# データセットの設定（Hugging Faceの公開データセット） - はじめ
#「fujiki/japanese_alpaca_data」（データセットの量：52,002）を使う場合の例
#（注：無料枠のGPU　- T4 GPUでは学習回数は1回〜数回程度かもしれません）
# データセットは最終的に 「instruction」「input」「output」の列になるように
# 前処理をしてください

# @markdown  Hugging Faceの該当データセットの名前を指定します。
llm_hf_dataset = "fujiki/japanese_alpaca_data" # @param {type:"string"}
dataset = load_dataset(llm_hf_dataset, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

# データセットの設定（Hugging Faceの公開データセット） - おわり



# 【おまけのコード】
# データセットの設定（Hugging Faceの公開データセット） - はじめ
#「izumi-lab/llm-japanese-dataset」（データセットの量：9,074,340）を使う場合の例
#（注：この規模だと無料枠のGPU　- T4 GPUではきついかもしれません）
# データセットは最終的に 「instruction」「input」「output」の列になるように
#  前処理をしてください
# Hugging Faceの既存のデータセットを使いたい場合には以下のコードを有効化します

# dataset = load_dataset("izumi-lab/llm-japanese-dataset", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

# データセットの設定（Hugging Faceの公開データセット） - おわり

**【データセットの例】**<br>
・[fujiki/japanese_alpaca_data（cc-by-nc-sa-4.0）| Hugging Face](https://huggingface.co/datasets/fujiki/japanese_alpaca_data)<br>
　データセット量：52,002<br><br>
・[izumi-lab/llm-japanese-dataset（cc-by-sa-4.0）| Hugging Face](https://huggingface.co/datasets/izumi-lab/llm-japanese-dataset)<br>
　データセット量：9,074,340<br><br>
データセットは最終的に 「**instruction**」「**input**」「**output**」の列になるように前処理をしてください。<br><br>

**【Hugging Faceで現在公開されている日本語データセットをチェック】**<br>
[キーワード「japanese」（トレンド順）- Datasets | Hugging Face](https://huggingface.co/datasets?sort=trending&search=japanese)
<br><br><br>


#【事前準備②：ファインチューニングのパラメータの指定】
「**実行コード①**」（LoRAの設定）は、1回のみ実行可能です。2回目の実行は反映されません。<br>
「**実行コード②**」（その他のパラメータの設定）は、何度でも実行可能です。<br>
各種パラメータの詳細をコード内に記載しておきました。<br>
必要に応じて「**コードの表示**」をクリックして、設定を変更してみてください。

In [ ]:
# @title 実行コード①（LoRAのパラメータの設定）

# LoRA（Low-Rank Adaptation）の設定
# 少ない学習パラメータで調整(全ての重みではなく、近似した小規模の行列を調整）
model = FastLanguageModel.get_peft_model(
    # LLMモデルの指定
    model,
    # LoRA Rディメンション（次元）の指定
    # 更新行列のランクを表すパラメータ
    # 一般的にrが小さいと、より短時間で計算量が少なくなる。大きいほど、更新行列は元の重みに近くなるが計算量が増える
    # ここの値を大きくするとファインチューニングを行うパラメータの割合を増やすことができます
    r = 16, # デフォルト 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    # LoRAを適用するモジュールの指定
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # LoRAアルファの指定
    # LoRAのスケーリングに使用されるパラメータ。
    # 更新行列の大きさを制限し、LoRAの過学習などを抑制
    lora_alpha = 16,
    # LoRAのドロップアウト率の指定
    # 更新行列の一部を無効化し、LoRAの過学習などを抑制
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # メモリの最適化 - 勾配チェックポインティング
    # 「per_device_train_batch_size = 1」でもGPUメモリがオーバーしてしまう時に、メモリを最適化する方法
    # 使用する場合には「use_gradient_checkpointing = True」に設定
    use_gradient_checkpointing = True,
    # ランダムシード値の指定
    random_state = 3407,
    # Rank-Stabilized LoRAの設定
    # 有効化したい場合には「use_rslora = True」に設定
    use_rslora = False,  # We support rank stabilized LoRA
    # LoRA-Fine-Tuning-Aware Quantizationの設定
    # 有効化したい場合には「loftq_config = loftq_config」に設定
    loftq_config = None, # And LoftQ
)

In [ ]:
# @title 実行コード②（その他のパラメータの設定）
# ファインチューニング用に各種インポート
from trl import SFTTrainer
from transformers import TrainingArguments

# ファインチューニングのその他のパラメータの設定
trainer = SFTTrainer(
    # LLMモデルの指定
    model = model,
    # トークナイザの指定
    tokenizer = tokenizer,
    # 学習用のデータセットを指定
    train_dataset = dataset,
    # 学習用のデータセットのテキストフィールドの指定
    dataset_text_field = "text",
    # 最大シーケンス長の指定
    max_seq_length = max_seq_length,
    # データのトークン化に使用するワーカー数の指定
    dataset_num_proc = 2,
    # シークエンスパッキングの設定
    # 短いシークエンスであれば、トレーニングを5倍速くすることができる
    # 「True」で有効化
    packing = False,
    args = TrainingArguments(
        # バッチサイズの指定
        per_device_train_batch_size = 2,
        # メモリの最適化：勾配累積のステップ数（エポック数）の指定
        # バッチ全体の勾配を一度に計算せずに、小さなバッチサイズで計算したものを集約しでバッチサイズを増やす
        gradient_accumulation_steps = 4,
        # 学習率をウォームアップするステップ数（エポック数）を指定
        # 「warmup_epochs=0」で0から増加
        warmup_steps = 5,
        # ファインチューニングの学習回数（ステップ数・エポック数）を指定
        # @markdown ファインチューニングの学習回数を指定します。
        max_steps = 30 # @param {type:"integer"}
        ,
        # 学習率の指定
        # 学習の際に重みを更新する際に使用する割合
        # 過学習（過剰適合・オーバーフィッティング）と学習不足（未学習・アンダーフィット）のバランスをとるための設定
        learning_rate = 2e-4,
        # FP16（16ビット浮動小数点）の使用有無を指定。「not」を削除すると有効化
        # LLMの重み（パラメータ）を16ビット精度 (FP16) でロード「float16 量子化」
        fp16 = not torch.cuda.is_bf16_supported(),
        # BFP16（Brain Float Point 16）の使用有無を指定。「= not torch.cuda.is_bf16_supported()」で無効化
        # LLMの重み（パラメータ）を16ビット精度 (BFP16) でロード「bfloat16 量子化」
        bf16 = torch.cuda.is_bf16_supported(),
        # 学習ログを記録する頻度を指定
        logging_steps = 1,
        # 最適化アルゴリズムにAdamW 8ビットを利用
        optim = "adamw_8bit",
        # 重み減衰の指定
        # 値を0以外（例：0.01）にすると、L2正規化が働き過学習の抑制効果
        weight_decay = 0.01,
        # 学習率のスケジュールの定義
        # 「linear」（線形の学習率を適用）
        lr_scheduler_type = "linear",
        # ランダムシード値の指定
        seed = 3407,
        # 結果の出力先
        output_dir = "outputs",
    ),
)


<br><br><br>

#ファインチューニングの実行

In [ ]:
# @title 実行コード
# メモリのステータスの表示設定
# （0番目の）GPUの情報を取得
gpu_stats = torch.cuda.get_device_properties(0)
# 予約メモリの最大GPUメモリ確保量を小数点以下3桁でGB表示
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# 使用するGPUのGPUメモリ容量を小数点以下3桁でGB表示
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# GPU名とメモリ容量を出力
print(f"GPU = {gpu_stats.name}. 最大メモリ = {max_memory} GB.")
# 確保しているGPUメモリの量を出力
print(f"予約メモリ = {start_gpu_memory} GBを確保")


# ファインチューニングの実行
trainer_stats = trainer.train()


# ファインチューニングに要した時間・GPUのの統計情報の表示設定
# 予約メモリの最大GPUメモリ確保量を小数点以下3桁でGB表示
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# ファインチューニングに要した最大GPUメモリを小数点以下3桁でGB表示
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# 使用したGPUのメモリ容量に対するファインチューニングで予約メモリの割合を計算し小数点以下3桁でGB表示
used_percentage = round(used_memory         /max_memory*100, 3)
# 使用したGPUのメモリ容量に対するファインチューニングに要したメモリ消費量の割合を計算し小数点以下3桁でGB表示
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
# ファインチューニングに要した時間を出力
print(f"学習時間 = {trainer_stats.metrics['train_runtime']} （秒）.")
print(f"学習時間 = {round(trainer_stats.metrics['train_runtime']/60, 2)} （分）")
print(f"予約メモリの最大GPUメモリ = {used_memory} GB.")
print(f"学習に要した予約メモリの最大GPUメモリ = {used_memory_for_lora} GB.")
print(f"GPUのメモリ使用率（予約メモリ/GPU容量） = {used_percentage} %.")
print(f"GPUのメモリ使用率（学習に要したメモリ消費量/GPU容量） = {lora_percentage} %.")

<br><br><br>

# ファインチューニングしたLLMで推論
それでは、先程ファインチューニングした学習モデルを使って推論をしてみましょう。

# 推論①：指示形式

In [ ]:
# @title 実行コード

alpaca_prompt = """以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。\n\n

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)
# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt = "こんにちは。" # @param {type:"string"}
inputs = tokenizer(
[
    alpaca_prompt.format(
        # instruction：プロンプトとして活用しています。
         input_prompt,
        # input：必要に応じて入力します。
        "",
        # output：文章を生成するにはここを空欄にします。
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64) #「max_new_tokens = 」の値で出力のトークン数を指定できます

In [ ]:
# @title 実行コード

alpaca_prompt = """以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。\n\n

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)
# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt = "YouTubeのRehabC – デジタルで、遊ぶ。チャンネルについて教えてください。" # @param {type:"string"}
inputs = tokenizer(
[
    alpaca_prompt.format(
        # instruction：プロンプトとして活用しています。
         input_prompt,
        # input：必要に応じて入力します。
        "",
        # output：文章を生成するにはここを空欄にします。
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64) #「max_new_tokens = 」の値で出力のトークン数を指定できます

<br><br>

# 推論②：対話型・チャット形式テンプレート

In [ ]:
# @title 実行コード

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    # サポートしているテンプレート「zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth」
    chat_template = "alpaca",
    # ShareGPTスタイル
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    # <|im_end|>を</s>に変換しマッピング
    map_eos_token = True,
)

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)

# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt_chat = "こんにちは。" # @param {type:"string"}
messages = [
    {"from": "human", "value": input_prompt_chat},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True) #「max_new_tokens = 」の値で出力のトークン数を指定できます
tokenizer.batch_decode(outputs)

In [ ]:
# @title 実行コード

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    # サポートしているテンプレート「zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth」
    chat_template = "alpaca",
    # ShareGPTスタイル
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    # <|im_end|>を</s>に変換しマッピング
    map_eos_token = True,
)

# 通常の2倍のスピードで推論を実行
FastLanguageModel.for_inference(model)

# @markdown 推論をするには、以下のプロンプトを入力後にコードを実行します。
input_prompt_chat = "YouTubeのRehabC – デジタルで、遊ぶ。チャンネルについて教えてください。" # @param {type:"string"}
messages = [
    {"from": "human", "value": input_prompt_chat},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True) #「max_new_tokens = 」の値で出力のトークン数を指定できます
tokenizer.batch_decode(outputs)

<br><br><br>

#【おまけコード】ファインチューニングしたLLMをGGUF形式にする前に保存したい場合
必要な箇所のコードの「**False**」の記載を「**True**」に変更後にコードを実行します。<br>
「**4-bit量子化としてLLMの結果を保存**」したい場合には<br>
「**if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit_forced",)**」<br>
にコードを変更後にコードを実行します。<br><br>
不必要な場合には、ここのコードを飛ばしてください。


In [ ]:
# @title 実行コード

# 16-bit量子化としてLLMの結果を保存
# ただ、この後実行するGGUF化の際に、処理の経過として16-bit量子化のLLMが出力されます
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
# 16bit量子化としてLLMの結果を保存したものをHugging Faceにプッシュ
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 4-bit量子化としてLLMの結果を保存
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit_forced",)
# 4-bit量子化としてLLMの結果を保存したものをHugging Faceにプッシュ
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit_forced", token = "")

# LoRAアダプターのみ保存
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
# LoRAアダプターのみ保存保存したものをHugging Faceにプッシュ
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

<br><br><br>

#ファインチューニングしたLLMを「量子化」+「GGUF形式」に変換
ローカル環境のアプリで使うために、QLoRAしたLLMをGGUF形式に変換して保存します。<br>
変換の過程で<br><br>
**・float 16のLLMが「model」フォルダ内に出力**<br>
（一例：12GB台）<br>

・**float 16の「GGUF形式」に変換された「model-unsloth.F16.gguf」ファイルが出力**<br>
（一例：12GB台）<br><br>
されます。<br>
その後、最終的に指定した量子化手法の形式のファイルが出力されます。<br>
一例：「**q2_k**」の量子化手法の場合、最終的に「**model-unsloth.Q2_K.gguf**」（2GB台）ファイルが出力されます。<br>
一例：「**q4_k_m**」の量子化手法の場合、最終的に「**model-unsloth.Q4_K_M.gguf**」（3〜4GB台）ファイルが出力されます。<br><br>
（一連の処理を実行するのに20分前後かかります）

In [ ]:
# @title 実行コード

# 「q4_k_m」（4-bit量子化）でGGUF形式に保存したい場合に利用
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# 以下のコードは、Hagging Faceにプッシュしたい場合に利用
# 「False」を「True」にすると有効化されます
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

**【参考情報：対応している量子化の手法の一例】**<br>（float 32/bfloat 32の際に26GBとなる7Bモデルの例・型の区分は容量で区分した大まかな分類）<br><br>
「**quantization_method =**」のコード内に以下の量子化の手法を記入し、上記のコードを実行。<br>
「**q2_k**」を使う場合には半角英数で「**quantization_method = "q2_k"**」と記入します。<br><br>
**・q2_k**（2-bit量子化）<br>
：最小型（一例：2.63GB）。<br>
attention.vwとfeed_forward.w2のテンソルにはQ4_Kを使用し、その他のテンソルにはQ2_Kを使用する。<br>＊注「K」：k-quantメソッドという量子化モデル
<br>
**・q3_k_s**（3-bit量子化）<br>
：超小型（一例：2.75GB）。<br>
すべてのテンソルにQ3_Kを使用する。<br>
**・q3_k_m**（3-bit量子化）<br>
 ：超小型（一例：3.07GB）。<br>
 attention.wv、attention.wo、feed_forward.w2のテンソルにはQ4_Kを使用し、それ以外のテンソルにはQ3_Kを使用する。<br>
**・q3_k_l**（3-bit量子化）<br>
：小型（一例：3.35GB）。<br>
attention.wv、attention.wo、feed_forward.w2のテンソルにはQ5_Kを使用し、それ以外のテンソルにはQ3_Kを使用する。<br>
**・q4_0**（4-bit量子化）<br>
：小型（一例：3.56G）。<br>
オリジナル版の4-bit量子化を使用する。<br>
**・q4_k_s**（4-bit量子化）<br>
：小型（一例：3.59GB）。<br>
すべてのテンソルにQ4_Kを使用する。<br>
**・q4_k_m【推奨】**（4-bit量子化）<br>
：中型 (一例：3.80GB)<br>
attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用する。それ以外はQ4_Kを使用する。<br>
**・q4_1**（4-bit量子化）<br>
：中型（一例：3.90GB）。<br>
q4_0よりは精度が高いが、q5_0ほどではない。しかし、q5モデルよりも推論が速い。<br>
**・q5_0**（5-bit量子化）<br>
：中型（一例：4.33GB）。<br>
高精度だが、リソースの使用量が多いため、低速の推論となる。<br>
**・q5_k_s**（5-bit量子化）<br>
：大型（一例：4.33GB）。<br>
すべてのテンソルにQ5_Kを使用する。<br>
**・q5_k_m【推奨】**（5-bit量子化）<br>
：大型（一例：4.45GB）。<br>
attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用する。それ以外はQ5_Kを使用する。<br>
**・q5_1**（5-bit量子化）<br>
：大型（一例：4.7GB）。<br>
q5_0よりも、さらに高い精度だが、リソースの使用量が多いため、さらに低速の推論となる。<br>
**・q6_k**（6-bit量子化）<br>
：超大型（一例：5.15GB）。<br>
すべてのテンソルにQ8_Kを使用する。<br>
**・q8_0**（8-bit量子化）<br>
：超大型（一例：6.7GB）。<br>
float16とほとんど区別がつかないレベルだが、リソースの使用量が多いため、低速の推論となる。ほとんどのユーザーにはお勧めできない。<br>
<br>
**【参考】**<br>
・[quantize.cpp - ggerganov/llama.cpp | GitHub](https://github.com/ggerganov/llama.cpp/blob/master/examples/quantize/quantize.cpp)<br>
（各量子化手法の容量）
<br>
・[save.py - unslothai/unsloth](https://github.com/unslothai/unsloth/blob/main/unsloth/save.py)<br>
（量子化のバリエーションと説明）<br><br><br>




# Google DriveのマウントとGGUF形式ファイルのダウンロード
 [Google Drive](https://www.google.com/intl/ja_ALL/drive/)  をマウントし、「**GGUF形式ファイル**」（例：model-unsloth.Q4_K_M.gguf）を「**MyDrive**」に移動後に、お使いのパソコンのローカル環境にダウンロードします。<br><br><br>

In [ ]:
# @title 実行コード
from google.colab import drive
drive.mount('/content/drive')

<br><br><br>

#【おまけコード】Janで使う前に量子化したGGUFのLLMで検証

必要に応じて、以下のチュートリアルコード<br><br>
[llama-cpp-python-for-Japanese-AI-Beginners.ipynb（The MIT License）| Google Colaboratory](https://colab.research.google.com/drive/1yZ1KCToAmKTSgWjjVpLzF48gwoHOq2yx?usp=sharing)<br><br>
を参考に、このノートブックに「**手順①**」「**手順③**」のコードをコピー&ペーストしてコードを実行し、量子化したGGUF形式のLLMの回答の精度を検証してみてください。<br><br><br>


<br><br><br>

# 【ステップ３】ローカル環境でファインチューニングしたLLMを実行 編
---
それでは、ローカル環境のアプリで量子化＋GGUF化したLLMを実行してみましょう。<br><br>
まずは、以下のリンク先<br><br>
**ChatGPT代替AIアプリをダウンロード：**<br>
[Jan：Open-source ChatGPT alternative（AGPLv3 License.）| Jan AI  ](https://jan.ai/)<br>
（対応OS：Windows・M1/M2/M3 Mac・Intel Mac・Linux AppImage/deb）<br><br>
のページで、現在お使い中のOSのバージョンのJanのアプリをダウンロードします。<br>
Janのアプリで、今回ファインチューニングしたLLMを使えるようにする方法は、以下の記事<br><br>

**Jan公式の解説記事：**<br>
[Import Models Manually | Jan AI](https://jan.ai/guides/using-models/import-manually/)<br>
（LLMを手動でアプリ内にインポートする方法）<br>
2024年3月23日時点では編集中になっていました...
<br><br>
を参照ください。<br>
インポートの手順例やアプリの使い方は、チュートリアル動画で解説予定です。<br><br>
**【追記：2024年3月23日】**<br>
アプリがアップデートされ簡単にGGUF形式のモデルをインポートできるようになりました。<br>
「**Settings**」の「**My Models**」の表示画面の右上にある「**Import Model**」をクリック後に表示されるウインドウに、ドラッグ&ドロップするとインポートできます。<br>
チュートリアル動画では「**Move model binary file**」を選択し、Janのアプリ内にモデルのファイルを複製させて使う方法を実行しています。
<br><br><br>
また、このノートブックで「**ELYZA-japanese-Llama-2-7b**」をファインチューニングしたものをJanのアプリで使えなかったため、Hugging Faceに公開してくださっている<br><br>

・[elyza/ELYZA-japanese-Llama-2-7b-fast-instruct | Hugging Face](https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast)<br><br>

の量子化版LLM<br><br>
・[mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-gguf | Hugging Face](https://huggingface.co/mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-gguf)<br><br>
のGGUF形式ファイルをJanで使う方法も解説予定です。<br><br>
**【追記：2024年3月25日】**<br>
チュートリアル動画を公開しました。<br>
Janのアプリにファインチューニング後にGGUF化したLLMをインポートする方法や、ELYZAをJanのアプリで使う方法は、動画を参照いただけますと幸いです。


# 【コピー & ペースト用素材】
**①Swallow用**<br>
**【Assistant】Instructions**<br><br>
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。
<br><br>
**②ELYZA用**<br>
**【Assistant】Instructions**<br><br>
あなたは誠実で優秀な日本人のアシスタントです。
<br><br>
**【Model Parameters】Prompt template**

In [ ]:
[INST] <<SYS>>\n{system_message}<</SYS>>\n{prompt}[/INST]

<br><br>**【チュートリアル記事】**<br>
<br>
：[【QLoRA編】日本語LLMのファインチューニング & 低スペックのローカル環境のアプリで動かす by 子供プログラマー](https://child-programmer.com/llm-ft-qlora-tutorial/)<br>


<br><br>【公開日】2024年3月17日<br>
【最終更新】2024年3月31日<br>
<br><br>

<br><br>
[日本人のための人工知能プログラマー入門講座（機械学習）by 子供プログラマー](https://child-programmer.com/ai/)